In [122]:
import numpy as np
import pandas as pd
import pulp
import itertools



In [125]:
# customer count ('0' is depot) 
customer_count = 10

In [126]:
# the number of vehicle
vehicle_count = 4

In [127]:
# the capacity of vehicle
vehicle_capacity = 50

In [139]:
# set depot x y coordinate
depot_x = 0
depot_y = 0

In [146]:
# make dataframe which contains customer points and demand of each customer
df = pd.DataFrame({"x":np.random.randint(2,20,customer_count), 
                   "y":np.random.randint(2,20,customer_count), 
                   "demand":np.random.randint(10, 20, customer_count)})

In [147]:
# set the depot as the origin (0,0)
df.iloc[0,0] = depot_x
df.iloc[0,1] = depot_y
df.iloc[0,2] = 0

In [148]:
df

,x,y,demand
0,0,0,0
1,19,15,16
2,10,16,14
3,11,6,13
4,14,11,17
5,18,5,10
6,4,9,14
7,5,9,18
8,13,2,17
9,7,5,11


In [153]:
len(df)

10

In [183]:
def distance_calculator(df):
    """
    The function to calculate the distance between each of the depot and customers
    """
    dist=np.zeros((len(df),len(df)))
    for i in range(len(df)):
        for j in range(len(df)):
            a=np.array((df.iloc[i]["x"],df.iloc[i]["y"]))
            b=np.array((df.iloc[j]["x"],df.iloc[j]["y"]))
            dist[i][j]=np.linalg.norm(a-b)
    return dist

In [184]:
distance=distance_calculator(df)

In [185]:
distance

array([[ 0.        , 24.20743687, 18.86796226, 12.52996409, 17.80449381,
        18.68154169,  9.8488578 , 10.29563014, 13.15294644,  8.60232527],
       [24.20743687,  0.        ,  9.05538514, 12.04159458,  6.40312424,
        10.04987562, 16.15549442, 15.23154621, 14.31782106, 15.62049935],
       [18.86796226,  9.05538514,  0.        , 10.04987562,  6.40312424,
        13.60147051,  9.21954446,  8.60232527, 14.31782106, 11.40175425],
       [12.52996409, 12.04159458, 10.04987562,  0.        ,  5.83095189,
         7.07106781,  7.61577311,  6.70820393,  4.47213595,  4.12310563],
       [17.80449381,  6.40312424,  6.40312424,  5.83095189,  0.        ,
         7.21110255, 10.19803903,  9.21954446,  9.05538514,  9.21954446],
       [18.68154169, 10.04987562, 13.60147051,  7.07106781,  7.21110255,
         0.        , 14.56021978, 13.60147051,  5.83095189, 11.        ],
       [ 9.8488578 , 16.15549442,  9.21954446,  7.61577311, 10.19803903,
        14.56021978,  0.        ,  1.        

![Cat](img/formula.jpg)

In [186]:
# solve with pulp

In [191]:
for vehicle_count in range(1,vehicle_count+1):
    # definition of LpProblem instance
    problem = pulp.LpProblem("CVRP", pulp.LpMinimize)
    
    # definition of variables which are 0/1
    x = [[[pulp.LpVariable("x%s_%s,%s"%(i,j,k), cat="Binary") if i != j else None for k in range(vehicle_count)]for j in range(customer_count)] for i in range(customer_count)]
    
    # add objective function
    problem += pulp.lpSum(distance[i][j] * x[i][j][k] if i != j else 0
                          for k in range(vehicle_count) 
                          for j in range(customer_count) 
                          for i in range (customer_count))
    # constraints
    # formula (2) only one visit of vehicle to the customer location
    for j in range(1, customer_count):
        problem += pulp.lpSum(x[i][j][k] if i != j else 0 
                              for i in range(customer_count) 
                              for k in range(vehicle_count)) == 1 
    # formula (3) all the vehicles depart from depot
    for k in range(vehicle_count):
        problem += pulp.lpSum(x[0][j][k] for j in range(1,customer_count)) == 1
        problem += pulp.lpSum(x[i][0][k] for i in range(1,customer_count)) == 1
    
    # formula(4) number of vehicles coming in and out of customers location is the same.
    for k in range(vehicle_count):
        for j in range(customer_count):
            problem += pulp.lpSum(x[i][j][k] if i != j else 0 
                                  for i in range(customer_count)) -  pulp.lpSum(x[j][i][k] for i in range(customer_count)) == 0
        
     #formula (5) the delivery capacity of each vehicle should not exceed the maximum capacity
    for k in range(vehicle_count):
        problem += pulp.lpSum(df.demand[j] * x[i][j][k] if i != j else 0 for i in range(customer_count) for j in range (1,customer_count)) <= vehicle_capacity 
    
    # formula (6) the constraint to remove subtours
    subtours = []
    for i in range(2,customer_count):
         subtours += itertools.combinations(range(1,customer_count), i)
    
    for s in subtours:
        problem += pulp.lpSum(x[i][j][k] if i !=j else 0 for i, j in itertools.permutations(s,2) for k in range(vehicle_count)) <= len(s) - 1
    
    print (problem)
    # print vehicle_count which needed for solving problem
    # print calculated minimum distance value
    if problem.solve() == 1:
        print('Vehicle Requirements:', vehicle_count)
        print('Moving Distance:', pulp.value(problem.objective))
        break

CVRP:
MINIMIZE
24.20743687382041*x0_1,0 + 18.867962264113206*x0_2,0 + 12.529964086141668*x0_3,0 + 17.804493814764857*x0_4,0 + 18.681541692269406*x0_5,0 + 9.848857801796104*x0_6,0 + 10.295630140987*x0_7,0 + 13.152946437965905*x0_8,0 + 8.602325267042627*x0_9,0 + 24.20743687382041*x1_0,0 + 9.055385138137417*x1_2,0 + 12.041594578792296*x1_3,0 + 6.4031242374328485*x1_4,0 + 10.04987562112089*x1_5,0 + 16.15549442140351*x1_6,0 + 15.231546211727817*x1_7,0 + 14.317821063276353*x1_8,0 + 15.620499351813308*x1_9,0 + 18.867962264113206*x2_0,0 + 9.055385138137417*x2_1,0 + 10.04987562112089*x2_3,0 + 6.4031242374328485*x2_4,0 + 13.601470508735444*x2_5,0 + 9.219544457292887*x2_6,0 + 8.602325267042627*x2_7,0 + 14.317821063276353*x2_8,0 + 11.40175425099138*x2_9,0 + 12.529964086141668*x3_0,0 + 12.041594578792296*x3_1,0 + 10.04987562112089*x3_2,0 + 5.830951894845301*x3_4,0 + 7.0710678118654755*x3_5,0 + 7.615773105863909*x3_6,0 + 6.708203932499369*x3_7,0 + 4.47213595499958*x3_8,0 + 4.123105625617661*x3_9,0 +